# Chapter 18: Testing and Evaluation - Solutions
**From: Zero to AI Agent**

**Try the exercises in the main notebook first before viewing solutions!**

---
## Section 18.1 Solutions

### Exercise 18.1.1: Testing a Validation Tool

In [ ]:
# File: exercise_1_18_1_solution.py
# Description: Exercise 1 Solution - Email validation tool with comprehensive tests

import pytest
import re


# The tool function
def validate_email(email: str) -> dict:
    """Validate an email address and return detailed results."""
    if not email or not isinstance(email, str):
        return {
            "valid": False,
            "error": "Email must be a non-empty string",
            "normalized": None
        }
    
    # Normalize the email
    normalized = email.strip().lower()
    
    # Basic regex pattern for email validation
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    if not re.match(pattern, email.strip()):
        return {
            "valid": False,
            "error": "Invalid email format",
            "normalized": normalized
        }
    
    # Check for common issues
    if ".." in normalized:
        return {
            "valid": False,
            "error": "Email cannot contain consecutive dots",
            "normalized": normalized
        }
    
    return {
        "valid": True,
        "error": None,
        "normalized": normalized
    }


# The tests
class TestValidateEmail:
    """Tests for the validate_email function."""
    
    def test_valid_simple_email(self):
        """Test a standard valid email."""
        result = validate_email("user@example.com")
        
        assert result["valid"] is True
        assert result["error"] is None
        assert result["normalized"] == "user@example.com"
    
    def test_valid_email_with_plus(self):
        """Test email with plus addressing."""
        result = validate_email("user+tag@example.com")
        
        assert result["valid"] is True
        assert result["normalized"] == "user+tag@example.com"
    
    def test_valid_email_with_dots(self):
        """Test email with dots in local part."""
        result = validate_email("first.last@example.com")
        
        assert result["valid"] is True
    
    def test_valid_email_normalized_to_lowercase(self):
        """Test that emails are normalized to lowercase."""
        result = validate_email("User@EXAMPLE.COM")
        
        assert result["valid"] is True
        assert result["normalized"] == "user@example.com"
    
    def test_valid_email_with_subdomain(self):
        """Test email with subdomain."""
        result = validate_email("user@mail.example.com")
        
        assert result["valid"] is True
    
    def test_invalid_no_at_symbol(self):
        """Test email without @ symbol."""
        result = validate_email("userexample.com")
        
        assert result["valid"] is False
        assert result["error"] == "Invalid email format"
    
    def test_invalid_no_domain(self):
        """Test email without domain."""
        result = validate_email("user@")
        
        assert result["valid"] is False
    
    def test_invalid_no_tld(self):
        """Test email without TLD."""
        result = validate_email("user@example")
        
        assert result["valid"] is False
    
    def test_invalid_consecutive_dots(self):
        """Test email with consecutive dots."""
        result = validate_email("user..name@example.com")
        
        assert result["valid"] is False
        assert "consecutive dots" in result["error"]
    
    def test_empty_string(self):
        """Test empty string input."""
        result = validate_email("")
        
        assert result["valid"] is False
        assert "non-empty string" in result["error"]
        assert result["normalized"] is None
    
    def test_none_input(self):
        """Test None input."""
        result = validate_email(None)
        
        assert result["valid"] is False
        assert result["normalized"] is None
    
    def test_whitespace_trimmed(self):
        """Test that whitespace is trimmed."""
        result = validate_email("  user@example.com  ")
        
        assert result["valid"] is True
        assert result["normalized"] == "user@example.com"


### Exercise 18.1.2: Testing State Helpers

In [ ]:
# File: exercise_2_18_1_solution.py
# Description: Exercise 2 Solution - Shopping cart state helpers with tests

import pytest
from typing import TypedDict


# State definitions
class CartItem(TypedDict):
    name: str
    quantity: int
    price: float


class CartState(TypedDict):
    items: list[CartItem]
    total: float
    coupon_code: str | None


# Helper functions
def add_item(state: CartState, name: str, quantity: int, price: float) -> dict:
    """Add an item to the cart. Returns state update."""
    # Check if item already exists
    for item in state["items"]:
        if item["name"] == name:
            # Update quantity of existing item
            new_items = []
            for i in state["items"]:
                if i["name"] == name:
                    new_items.append({
                        "name": name,
                        "quantity": i["quantity"] + quantity,
                        "price": price
                    })
                else:
                    new_items.append(i)
            return {"items": new_items}
    
    # Add new item
    new_items = state["items"] + [{"name": name, "quantity": quantity, "price": price}]
    return {"items": new_items}


def remove_item(state: CartState, name: str) -> dict:
    """Remove an item from the cart. Returns state update."""
    new_items = [item for item in state["items"] if item["name"] != name]
    
    if len(new_items) == len(state["items"]):
        # Item wasn't found - return unchanged
        return {"items": state["items"]}
    
    return {"items": new_items}


def apply_coupon(state: CartState, code: str, valid_coupons: dict[str, float]) -> dict:
    """Apply a coupon code. Returns state update with discount info."""
    if code not in valid_coupons:
        return {"coupon_code": None, "coupon_error": "Invalid coupon code"}
    
    return {"coupon_code": code, "coupon_error": None}


def calculate_total(state: CartState, valid_coupons: dict[str, float] = None) -> dict:
    """Calculate cart total. Returns state update."""
    valid_coupons = valid_coupons or {}
    
    subtotal = sum(item["price"] * item["quantity"] for item in state["items"])
    
    discount = 0.0
    if state.get("coupon_code") and state["coupon_code"] in valid_coupons:
        discount = subtotal * valid_coupons[state["coupon_code"]]
    
    total = subtotal - discount
    return {"total": round(total, 2)}


# Tests
class TestAddItem:
    def test_add_new_item(self):
        state = {"items": [], "total": 0, "coupon_code": None}
        
        result = add_item(state, "Widget", 2, 9.99)
        
        assert len(result["items"]) == 1
        assert result["items"][0]["name"] == "Widget"
        assert result["items"][0]["quantity"] == 2
        assert result["items"][0]["price"] == 9.99
    
    def test_add_item_increases_quantity_if_exists(self):
        state = {
            "items": [{"name": "Widget", "quantity": 2, "price": 9.99}],
            "total": 0,
            "coupon_code": None
        }
        
        result = add_item(state, "Widget", 3, 9.99)
        
        assert len(result["items"]) == 1
        assert result["items"][0]["quantity"] == 5
    
    def test_add_different_items(self):
        state = {
            "items": [{"name": "Widget", "quantity": 2, "price": 9.99}],
            "total": 0,
            "coupon_code": None
        }
        
        result = add_item(state, "Gadget", 1, 19.99)
        
        assert len(result["items"]) == 2


class TestRemoveItem:
    def test_remove_existing_item(self):
        state = {
            "items": [
                {"name": "Widget", "quantity": 2, "price": 9.99},
                {"name": "Gadget", "quantity": 1, "price": 19.99}
            ],
            "total": 0,
            "coupon_code": None
        }
        
        result = remove_item(state, "Widget")
        
        assert len(result["items"]) == 1
        assert result["items"][0]["name"] == "Gadget"
    
    def test_remove_nonexistent_item_returns_unchanged(self):
        state = {
            "items": [{"name": "Widget", "quantity": 2, "price": 9.99}],
            "total": 0,
            "coupon_code": None
        }
        
        result = remove_item(state, "Nonexistent")
        
        assert len(result["items"]) == 1
        assert result["items"][0]["name"] == "Widget"
    
    def test_remove_from_empty_cart(self):
        state = {"items": [], "total": 0, "coupon_code": None}
        
        result = remove_item(state, "Widget")
        
        assert result["items"] == []


class TestApplyCoupon:
    def test_apply_valid_coupon(self):
        state = {"items": [], "total": 0, "coupon_code": None}
        valid_coupons = {"SAVE10": 0.10, "SAVE20": 0.20}
        
        result = apply_coupon(state, "SAVE10", valid_coupons)
        
        assert result["coupon_code"] == "SAVE10"
        assert result.get("coupon_error") is None
    
    def test_apply_invalid_coupon(self):
        state = {"items": [], "total": 0, "coupon_code": None}
        valid_coupons = {"SAVE10": 0.10}
        
        result = apply_coupon(state, "INVALID", valid_coupons)
        
        assert result["coupon_code"] is None
        assert "Invalid" in result["coupon_error"]


class TestCalculateTotal:
    def test_calculate_total_no_discount(self):
        state = {
            "items": [
                {"name": "Widget", "quantity": 2, "price": 10.00},
                {"name": "Gadget", "quantity": 1, "price": 20.00}
            ],
            "total": 0,
            "coupon_code": None
        }
        
        result = calculate_total(state)
        
        assert result["total"] == 40.00
    
    def test_calculate_total_with_discount(self):
        state = {
            "items": [{"name": "Widget", "quantity": 1, "price": 100.00}],
            "total": 0,
            "coupon_code": "SAVE10"
        }
        valid_coupons = {"SAVE10": 0.10}
        
        result = calculate_total(state, valid_coupons)
        
        assert result["total"] == 90.00
    
    def test_calculate_total_empty_cart(self):
        state = {"items": [], "total": 0, "coupon_code": None}
        
        result = calculate_total(state)
        
        assert result["total"] == 0.00


### Exercise 18.1.3: Mocking an API Tool

In [ ]:
# File: exercise_3_18_1_solution.py
# Description: Exercise 3 Solution - Weather API tool with async mocking tests

import pytest
from unittest.mock import AsyncMock, patch, MagicMock
import httpx


# The weather tool function
async def get_weather(city: str) -> dict:
    """Fetch current weather for a city from the weather API."""
    if not city or not city.strip():
        return {"error": "City name is required", "temperature": None}
    
    city = city.strip()
    
    try:
        async with httpx.AsyncClient(timeout=10.0) as client:
            response = await client.get(
                f"https://api.weather.example.com/current",
                params={"city": city}
            )
            
            if response.status_code == 404:
                return {"error": f"City '{city}' not found", "temperature": None}
            
            response.raise_for_status()
            data = response.json()
            
            return {
                "city": data["location"]["name"],
                "temperature": data["current"]["temp_c"],
                "condition": data["current"]["condition"]["text"],
                "error": None
            }
    
    except httpx.TimeoutException:
        return {"error": "Request timed out", "temperature": None}
    except httpx.HTTPStatusError as e:
        return {"error": f"API error: {e.response.status_code}", "temperature": None}
    except Exception as e:
        return {"error": f"Unexpected error: {str(e)}", "temperature": None}


# Tests
class TestGetWeather:
    
    @pytest.mark.asyncio
    async def test_successful_response(self):
        """Test correct parsing of successful API response."""
        mock_api_response = {
            "location": {"name": "London"},
            "current": {
                "temp_c": 15.5,
                "condition": {"text": "Partly cloudy"}
            }
        }
        
        with patch("httpx.AsyncClient") as MockClient:
            mock_client = AsyncMock()
            mock_response = MagicMock()
            mock_response.status_code = 200
            mock_response.json.return_value = mock_api_response
            mock_response.raise_for_status = MagicMock()
            mock_client.get.return_value = mock_response
            MockClient.return_value.__aenter__.return_value = mock_client
            
            result = await get_weather("London")
            
            assert result["city"] == "London"
            assert result["temperature"] == 15.5
            assert result["condition"] == "Partly cloudy"
            assert result["error"] is None
    
    @pytest.mark.asyncio
    async def test_city_not_found_404(self):
        """Test handling of 404 response."""
        with patch("httpx.AsyncClient") as MockClient:
            mock_client = AsyncMock()
            mock_response = MagicMock()
            mock_response.status_code = 404
            mock_client.get.return_value = mock_response
            MockClient.return_value.__aenter__.return_value = mock_client
            
            result = await get_weather("Nonexistentville")
            
            assert result["temperature"] is None
            assert "not found" in result["error"]
            assert "Nonexistentville" in result["error"]
    
    @pytest.mark.asyncio
    async def test_timeout_handling(self):
        """Test handling of network timeout."""
        with patch("httpx.AsyncClient") as MockClient:
            mock_client = AsyncMock()
            mock_client.get.side_effect = httpx.TimeoutException("Connection timed out")
            MockClient.return_value.__aenter__.return_value = mock_client
            
            result = await get_weather("Tokyo")
            
            assert result["temperature"] is None
            assert "timed out" in result["error"].lower()
    
    @pytest.mark.asyncio
    async def test_correct_city_passed_to_api(self):
        """Test that the city name is correctly passed to the API."""
        mock_api_response = {
            "location": {"name": "Paris"},
            "current": {"temp_c": 20.0, "condition": {"text": "Sunny"}}
        }
        
        with patch("httpx.AsyncClient") as MockClient:
            mock_client = AsyncMock()
            mock_response = MagicMock()
            mock_response.status_code = 200
            mock_response.json.return_value = mock_api_response
            mock_response.raise_for_status = MagicMock()
            mock_client.get.return_value = mock_response
            MockClient.return_value.__aenter__.return_value = mock_client
            
            await get_weather("Paris")
            
            # Verify the API was called with correct parameters
            mock_client.get.assert_called_once()
            call_kwargs = mock_client.get.call_args
            assert call_kwargs.kwargs["params"]["city"] == "Paris"
    
    @pytest.mark.asyncio
    async def test_empty_city_name(self):
        """Test handling of empty city name."""
        result = await get_weather("")
        
        assert result["temperature"] is None
        assert "required" in result["error"].lower()
    
    @pytest.mark.asyncio
    async def test_whitespace_city_name_trimmed(self):
        """Test that whitespace is trimmed from city name."""
        mock_api_response = {
            "location": {"name": "Berlin"},
            "current": {"temp_c": 18.0, "condition": {"text": "Clear"}}
        }
        
        with patch("httpx.AsyncClient") as MockClient:
            mock_client = AsyncMock()
            mock_response = MagicMock()
            mock_response.status_code = 200
            mock_response.json.return_value = mock_api_response
            mock_response.raise_for_status = MagicMock()
            mock_client.get.return_value = mock_response
            MockClient.return_value.__aenter__.return_value = mock_client
            
            await get_weather("  Berlin  ")
            
            call_kwargs = mock_client.get.call_args
            assert call_kwargs.kwargs["params"]["city"] == "Berlin"


---
## Section 18.2 Solutions

### Exercise 18.2.1: Testing a Multi-Node Workflow

In [ ]:
# File: exercise_1_18_2_solution.py
# Description: Exercise 1 Solution - Order processing workflow with integration tests

import pytest
from unittest.mock import MagicMock
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END


# State definition
class OrderState(TypedDict):
    order_id: str
    items: list[dict]
    subtotal: float | None
    tax: float | None
    total: float | None
    is_valid: bool | None
    error: str | None
    confirmation_message: str | None


# Node functions
def validate_order(state: OrderState) -> dict:
    """Validate order data."""
    errors = []
    
    if not state.get("order_id"):
        errors.append("Order ID is required")
    
    if not state.get("items") or len(state["items"]) == 0:
        errors.append("Order must have at least one item")
    
    for item in state.get("items", []):
        if item.get("quantity", 0) <= 0:
            errors.append(f"Invalid quantity for {item.get('name', 'unknown item')}")
        if item.get("price", 0) <= 0:
            errors.append(f"Invalid price for {item.get('name', 'unknown item')}")
    
    if errors:
        return {"is_valid": False, "error": "; ".join(errors)}
    
    return {"is_valid": True, "error": None}


def calculate_total(state: OrderState) -> dict:
    """Calculate order total with tax."""
    subtotal = sum(
        item["price"] * item["quantity"] 
        for item in state["items"]
    )
    tax_rate = 0.08  # 8% tax
    tax = round(subtotal * tax_rate, 2)
    total = round(subtotal + tax, 2)
    
    return {
        "subtotal": subtotal,
        "tax": tax,
        "total": total
    }


def confirm_order(state: OrderState) -> dict:
    """Generate order confirmation."""
    message = (
        f"Order {state['order_id']} confirmed! "
        f"Total: ${state['total']:.2f} (includes ${state['tax']:.2f} tax)"
    )
    return {"confirmation_message": message}


def route_after_validation(state: OrderState) -> str:
    """Route based on validation result."""
    if state.get("is_valid"):
        return "calculate_total"
    else:
        return END


def build_order_graph():
    """Build the order processing graph."""
    graph = StateGraph(OrderState)
    
    graph.add_node("validate_order", validate_order)
    graph.add_node("calculate_total", calculate_total)
    graph.add_node("confirm_order", confirm_order)
    
    graph.add_edge(START, "validate_order")
    graph.add_conditional_edges("validate_order", route_after_validation)
    graph.add_edge("calculate_total", "confirm_order")
    graph.add_edge("confirm_order", END)
    
    return graph.compile()


# Integration tests
class TestOrderProcessingWorkflow:
    """Integration tests for the complete order processing workflow."""
    
    @pytest.fixture
    def graph(self):
        return build_order_graph()
    
    def test_valid_order_flows_through_all_nodes(self, graph):
        """Test that valid orders complete the full workflow."""
        initial_state = {
            "order_id": "ORD-001",
            "items": [
                {"name": "Widget", "price": 10.00, "quantity": 2},
                {"name": "Gadget", "price": 25.00, "quantity": 1}
            ],
            "subtotal": None,
            "tax": None,
            "total": None,
            "is_valid": None,
            "error": None,
            "confirmation_message": None
        }
        
        result = graph.invoke(initial_state)
        
        # Verify validation passed
        assert result["is_valid"] is True
        assert result["error"] is None
        
        # Verify calculation happened
        assert result["subtotal"] == 45.00  # (10*2) + (25*1)
        assert result["tax"] == 3.60  # 45 * 0.08
        assert result["total"] == 48.60
        
        # Verify confirmation generated
        assert result["confirmation_message"] is not None
        assert "ORD-001" in result["confirmation_message"]
        assert "48.60" in result["confirmation_message"]
    
    def test_invalid_order_stops_at_validation(self, graph):
        """Test that invalid orders don't proceed to calculation."""
        initial_state = {
            "order_id": "ORD-002",
            "items": [],  # Empty items - invalid!
            "subtotal": None,
            "tax": None,
            "total": None,
            "is_valid": None,
            "error": None,
            "confirmation_message": None
        }
        
        result = graph.invoke(initial_state)
        
        # Validation should fail
        assert result["is_valid"] is False
        assert result["error"] is not None
        assert "at least one item" in result["error"]
        
        # Calculation should NOT have happened
        assert result["subtotal"] is None
        assert result["total"] is None
        
        # Confirmation should NOT have happened
        assert result["confirmation_message"] is None
    
    def test_state_accumulates_through_workflow(self, graph):
        """Test that state is correctly accumulated at each step."""
        initial_state = {
            "order_id": "ORD-003",
            "items": [{"name": "Test Item", "price": 100.00, "quantity": 1}],
            "subtotal": None,
            "tax": None,
            "total": None,
            "is_valid": None,
            "error": None,
            "confirmation_message": None
        }
        
        result = graph.invoke(initial_state)
        
        # Original data should be preserved
        assert result["order_id"] == "ORD-003"
        assert result["items"] == initial_state["items"]
        
        # Each stage should have added its data
        assert result["is_valid"] is not None  # From validate_order
        assert result["subtotal"] is not None  # From calculate_total
        assert result["confirmation_message"] is not None  # From confirm_order
    
    def test_missing_order_id_fails_validation(self, graph):
        """Test validation catches missing order ID."""
        initial_state = {
            "order_id": "",  # Empty order ID
            "items": [{"name": "Item", "price": 10.00, "quantity": 1}],
            "subtotal": None,
            "tax": None,
            "total": None,
            "is_valid": None,
            "error": None,
            "confirmation_message": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["is_valid"] is False
        assert "Order ID" in result["error"]
    
    def test_invalid_item_quantity_fails_validation(self, graph):
        """Test validation catches invalid quantities."""
        initial_state = {
            "order_id": "ORD-004",
            "items": [{"name": "Item", "price": 10.00, "quantity": -1}],
            "subtotal": None,
            "tax": None,
            "total": None,
            "is_valid": None,
            "error": None,
            "confirmation_message": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["is_valid"] is False
        assert "Invalid quantity" in result["error"]


### Exercise 18.2.2: Testing Branching Logic

In [ ]:
# File: exercise_2_18_2_solution.py
# Description: Exercise 2 Solution - Sentiment routing with integration tests

import pytest
from unittest.mock import MagicMock, AsyncMock
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage


class SentimentState(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str | None
    handler_invoked: str | None
    response: str | None


def classify_sentiment(state: SentimentState, llm) -> dict:
    """Classify the sentiment of the last message."""
    last_message = state["messages"][-1].content
    
    response = llm.invoke([HumanMessage(content=f"Classify sentiment: {last_message}")])
    sentiment = response.content.strip().lower()
    
    # Normalize to expected values
    if sentiment not in ["positive", "negative", "neutral"]:
        sentiment = "neutral"
    
    return {"sentiment": sentiment}


def route_by_sentiment(state: SentimentState) -> str:
    """Route to appropriate handler based on sentiment."""
    sentiment = state.get("sentiment", "neutral")
    
    if sentiment == "positive":
        return "handle_positive"
    elif sentiment == "negative":
        return "handle_negative"
    else:
        return "handle_neutral"


def handle_positive(state: SentimentState) -> dict:
    """Handle positive sentiment messages."""
    return {
        "handler_invoked": "positive",
        "response": "Thank you for your positive feedback! We're glad you're happy.",
        "messages": [AIMessage(content="Thank you for your positive feedback!")]
    }


def handle_negative(state: SentimentState) -> dict:
    """Handle negative sentiment messages."""
    return {
        "handler_invoked": "negative",
        "response": "We're sorry to hear you're unhappy. Let us help resolve this.",
        "messages": [AIMessage(content="We're sorry to hear that. How can we help?")]
    }


def handle_neutral(state: SentimentState) -> dict:
    """Handle neutral sentiment messages."""
    return {
        "handler_invoked": "neutral",
        "response": "Thank you for reaching out. How can we assist you today?",
        "messages": [AIMessage(content="How can we assist you today?")]
    }


def build_sentiment_graph(llm):
    """Build the sentiment routing graph."""
    graph = StateGraph(SentimentState)
    
    graph.add_node("classify_sentiment", lambda s: classify_sentiment(s, llm))
    graph.add_node("handle_positive", handle_positive)
    graph.add_node("handle_negative", handle_negative)
    graph.add_node("handle_neutral", handle_neutral)
    
    graph.add_edge(START, "classify_sentiment")
    graph.add_conditional_edges("classify_sentiment", route_by_sentiment)
    graph.add_edge("handle_positive", END)
    graph.add_edge("handle_negative", END)
    graph.add_edge("handle_neutral", END)
    
    return graph.compile()


# Integration tests
class TestSentimentRouting:
    """Integration tests for sentiment-based routing."""
    
    @pytest.fixture
    def mock_llm_positive(self):
        """Mock LLM that classifies as positive."""
        llm = MagicMock()
        llm.invoke.return_value = MagicMock(content="positive")
        return llm
    
    @pytest.fixture
    def mock_llm_negative(self):
        """Mock LLM that classifies as negative."""
        llm = MagicMock()
        llm.invoke.return_value = MagicMock(content="negative")
        return llm
    
    @pytest.fixture
    def mock_llm_neutral(self):
        """Mock LLM that classifies as neutral."""
        llm = MagicMock()
        llm.invoke.return_value = MagicMock(content="neutral")
        return llm
    
    def test_positive_sentiment_routes_to_positive_handler(self, mock_llm_positive):
        """Test that positive sentiment routes correctly."""
        graph = build_sentiment_graph(mock_llm_positive)
        
        initial_state = {
            "messages": [HumanMessage(content="I love your product!")],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["sentiment"] == "positive"
        assert result["handler_invoked"] == "positive"
        assert "positive feedback" in result["response"].lower()
    
    def test_negative_sentiment_routes_to_negative_handler(self, mock_llm_negative):
        """Test that negative sentiment routes correctly."""
        graph = build_sentiment_graph(mock_llm_negative)
        
        initial_state = {
            "messages": [HumanMessage(content="This is terrible!")],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["sentiment"] == "negative"
        assert result["handler_invoked"] == "negative"
        assert "sorry" in result["response"].lower()
    
    def test_neutral_sentiment_routes_to_neutral_handler(self, mock_llm_neutral):
        """Test that neutral sentiment routes correctly."""
        graph = build_sentiment_graph(mock_llm_neutral)
        
        initial_state = {
            "messages": [HumanMessage(content="What are your hours?")],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["sentiment"] == "neutral"
        assert result["handler_invoked"] == "neutral"
        assert "assist" in result["response"].lower()
    
    def test_unknown_sentiment_defaults_to_neutral(self):
        """Test that unrecognized sentiment defaults to neutral."""
        mock_llm = MagicMock()
        mock_llm.invoke.return_value = MagicMock(content="confused")
        
        graph = build_sentiment_graph(mock_llm)
        
        initial_state = {
            "messages": [HumanMessage(content="...")],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        result = graph.invoke(initial_state)
        
        # Should normalize to neutral
        assert result["sentiment"] == "neutral"
        assert result["handler_invoked"] == "neutral"
    
    def test_llm_is_called_with_user_message(self, mock_llm_positive):
        """Verify the LLM receives the user's message for classification."""
        graph = build_sentiment_graph(mock_llm_positive)
        
        test_message = "This is my specific test message"
        initial_state = {
            "messages": [HumanMessage(content=test_message)],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        graph.invoke(initial_state)
        
        # Verify LLM was called
        mock_llm_positive.invoke.assert_called_once()
        
        # Verify the message content was passed
        call_args = mock_llm_positive.invoke.call_args[0][0]
        assert any(test_message in str(msg) for msg in call_args)
    
    def test_response_message_added_to_state(self, mock_llm_positive):
        """Verify handler adds response to messages."""
        graph = build_sentiment_graph(mock_llm_positive)
        
        initial_state = {
            "messages": [HumanMessage(content="Great service!")],
            "sentiment": None,
            "handler_invoked": None,
            "response": None
        }
        
        result = graph.invoke(initial_state)
        
        # Should have original message plus AI response
        assert len(result["messages"]) >= 2
        
        # Last message should be from AI
        ai_messages = [m for m in result["messages"] if isinstance(m, AIMessage)]
        assert len(ai_messages) >= 1


### Exercise 18.2.3: Testing Conversation Context

In [ ]:
# File: exercise_3_18_2_solution.py
# Description: Exercise 3 Solution - FAQ agent with context maintenance

import pytest
from unittest.mock import MagicMock
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage


class FAQState(TypedDict):
    messages: Annotated[list, add_messages]
    topics_discussed: list[str]
    current_topic: str | None


def detect_topic(message: str, previous_topics: list[str]) -> str:
    """Detect the topic of a message, considering context."""
    message_lower = message.lower()
    
    # Direct topic mentions
    if "pricing" in message_lower or "cost" in message_lower or "price" in message_lower:
        return "pricing"
    elif "shipping" in message_lower or "delivery" in message_lower:
        return "shipping"
    elif "return" in message_lower or "refund" in message_lower:
        return "returns"
    elif "discount" in message_lower or "coupon" in message_lower or "sale" in message_lower:
        # Context-aware: "discount" after "pricing" is about pricing discounts
        if "pricing" in previous_topics:
            return "pricing"
        return "discounts"
    elif "hours" in message_lower or "open" in message_lower:
        return "hours"
    
    # If no direct match, continue previous topic
    if previous_topics:
        return previous_topics[-1]
    
    return "general"


def answer_question(state: FAQState, llm) -> dict:
    """Generate an answer based on topic and context."""
    messages = state["messages"]
    last_message = messages[-1].content
    topics_discussed = state.get("topics_discussed", [])
    
    # Detect topic with context awareness
    topic = detect_topic(last_message, topics_discussed)
    
    # Build context-aware prompt
    context = ""
    if topics_discussed:
        context = f"Previously discussed topics: {', '.join(topics_discussed)}. "
    
    prompt = f"{context}User question about {topic}: {last_message}"
    
    response = llm.invoke([
        SystemMessage(content=f"You are answering a question about {topic}."),
        HumanMessage(content=prompt)
    ])
    
    # Update topics discussed
    new_topics = topics_discussed.copy()
    if topic not in new_topics:
        new_topics.append(topic)
    
    return {
        "messages": [AIMessage(content=response.content)],
        "topics_discussed": new_topics,
        "current_topic": topic
    }


def build_faq_graph(llm):
    """Build the FAQ agent graph."""
    graph = StateGraph(FAQState)
    
    graph.add_node("answer", lambda s: answer_question(s, llm))
    
    graph.add_edge(START, "answer")
    graph.add_edge("answer", END)
    
    return graph.compile()


# Integration tests
class TestFAQContextMaintenance:
    """Integration tests for FAQ context maintenance."""
    
    @pytest.fixture
    def mock_llm(self):
        """Create a mock LLM that echoes the topic."""
        llm = MagicMock()
        # Return response that includes topic for verification
        def dynamic_response(messages):
            # Extract topic from system message
            system_msg = next((m for m in messages if isinstance(m, SystemMessage)), None)
            topic = "unknown"
            if system_msg:
                topic = system_msg.content.split("about ")[-1].rstrip(".")
            return MagicMock(content=f"Here's information about {topic}")
        
        llm.invoke.side_effect = dynamic_response
        return llm
    
    def test_first_question_starts_fresh_context(self, mock_llm):
        """Test that first question has no prior context."""
        graph = build_faq_graph(mock_llm)
        
        initial_state = {
            "messages": [HumanMessage(content="What is your pricing?")],
            "topics_discussed": [],
            "current_topic": None
        }
        
        result = graph.invoke(initial_state)
        
        assert result["current_topic"] == "pricing"
        assert "pricing" in result["topics_discussed"]
        assert len(result["topics_discussed"]) == 1
    
    def test_context_maintained_across_questions(self, mock_llm):
        """Test that context accumulates across questions."""
        graph = build_faq_graph(mock_llm)
        
        # First question about pricing
        state_1 = {
            "messages": [HumanMessage(content="What is your pricing?")],
            "topics_discussed": [],
            "current_topic": None
        }
        result_1 = graph.invoke(state_1)
        
        assert result_1["current_topic"] == "pricing"
        assert result_1["topics_discussed"] == ["pricing"]
        
        # Second question - simulate continuing conversation
        state_2 = {
            "messages": result_1["messages"] + [HumanMessage(content="What about shipping?")],
            "topics_discussed": result_1["topics_discussed"],
            "current_topic": result_1["current_topic"]
        }
        result_2 = graph.invoke(state_2)
        
        assert result_2["current_topic"] == "shipping"
        assert "pricing" in result_2["topics_discussed"]
        assert "shipping" in result_2["topics_discussed"]
    
    def test_discount_question_uses_pricing_context(self, mock_llm):
        """Test that 'discounts' after 'pricing' stays in pricing context."""
        graph = build_faq_graph(mock_llm)
        
        # First establish pricing context
        state_1 = {
            "messages": [HumanMessage(content="What are your prices?")],
            "topics_discussed": [],
            "current_topic": None
        }
        result_1 = graph.invoke(state_1)
        
        # Now ask about discounts - should relate to pricing
        state_2 = {
            "messages": result_1["messages"] + [HumanMessage(content="Any discounts available?")],
            "topics_discussed": result_1["topics_discussed"],
            "current_topic": result_1["current_topic"]
        }
        result_2 = graph.invoke(state_2)
        
        # Because pricing was discussed, discount should be treated as pricing-related
        assert result_2["current_topic"] == "pricing"
    
    def test_discount_question_without_context(self, mock_llm):
        """Test that 'discounts' without context is its own topic."""
        graph = build_faq_graph(mock_llm)
        
        # Ask about discounts first (no pricing context)
        initial_state = {
            "messages": [HumanMessage(content="Do you have any discounts?")],
            "topics_discussed": [],
            "current_topic": None
        }
        
        result = graph.invoke(initial_state)
        
        # Without pricing context, should be its own topic
        assert result["current_topic"] == "discounts"
    
    def test_ambiguous_followup_uses_previous_topic(self, mock_llm):
        """Test that ambiguous follow-ups use the previous topic."""
        graph = build_faq_graph(mock_llm)
        
        # Establish shipping topic
        state_1 = {
            "messages": [HumanMessage(content="How does shipping work?")],
            "topics_discussed": [],
            "current_topic": None
        }
        result_1 = graph.invoke(state_1)
        
        # Ask ambiguous follow-up
        state_2 = {
            "messages": result_1["messages"] + [HumanMessage(content="What about international?")],
            "topics_discussed": result_1["topics_discussed"],
            "current_topic": result_1["current_topic"]
        }
        result_2 = graph.invoke(state_2)
        
        # Should continue with shipping topic
        assert result_2["current_topic"] == "shipping"
    
    def test_messages_preserved_through_conversation(self, mock_llm):
        """Test that all messages are preserved in conversation."""
        graph = build_faq_graph(mock_llm)
        
        # Build up a multi-turn conversation
        state = {
            "messages": [HumanMessage(content="Question 1: What are your hours?")],
            "topics_discussed": [],
            "current_topic": None
        }
        result = graph.invoke(state)
        
        # Add second question
        state = {
            "messages": result["messages"] + [HumanMessage(content="Question 2: What about weekends?")],
            "topics_discussed": result["topics_discussed"],
            "current_topic": result["current_topic"]
        }
        result = graph.invoke(state)
        
        # All messages should be present
        all_content = " ".join(m.content for m in result["messages"])
        assert "Question 1" in all_content
        assert "Question 2" in all_content
        
        # Should have both user questions and AI responses
        human_messages = [m for m in result["messages"] if isinstance(m, HumanMessage)]
        ai_messages = [m for m in result["messages"] if isinstance(m, AIMessage)]
        
        assert len(human_messages) == 2
        assert len(ai_messages) == 2


---
## Section 18.3 Solutions

### Exercise 18.3.1: Custom Evaluation Rubric

In [ ]:
# File: exercise_1_18_3_solution.py
# Description: Exercise 1 Solution - Customer service evaluator with 4 criteria

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()


class CustomerServiceEvaluator:
    """
    Evaluate customer service agent responses across four criteria:
    - Accuracy: Factual correctness of information provided
    - Empathy: Emotional tone and understanding
    - Actionability: Does it help solve the problem?
    - Policy Compliance: Stays within company guidelines
    """
    
    def __init__(self, model: str = "gpt-4o"):
        self.llm = ChatOpenAI(model=model, temperature=0)
        
        # Define the rubric for each criterion
        self.rubrics = {
            "accuracy": """
Evaluate the ACCURACY of the response - is the information factually correct?

1 = Contains significant factual errors or misinformation
2 = Contains some inaccuracies that could mislead the customer
3 = Mostly accurate with minor errors that don't affect the outcome
4 = Accurate information with only trivial imprecisions
5 = Completely accurate, all facts verified and correct
""",
            "empathy": """
Evaluate the EMPATHY of the response - does it show understanding and care?

1 = Cold, dismissive, or robotic with no acknowledgment of customer feelings
2 = Minimal acknowledgment, feels scripted and impersonal
3 = Adequate acknowledgment but could be warmer
4 = Warm and understanding, makes customer feel heard
5 = Exceptional empathy, perfectly balances professionalism with genuine care
""",
            "actionability": """
Evaluate the ACTIONABILITY of the response - does it help solve the problem?

1 = No clear next steps, customer left confused about what to do
2 = Vague suggestions that don't directly address the issue
3 = Some helpful guidance but incomplete or unclear
4 = Clear, specific steps that should resolve the issue
5 = Comprehensive solution with alternatives and proactive help
""",
            "policy_compliance": """
Evaluate POLICY COMPLIANCE - does the response stay within appropriate boundaries?

1 = Violates policies (unauthorized promises, shares restricted info, etc.)
2 = Bends rules inappropriately or makes questionable commitments
3 = Follows policies but rigidly, missing opportunities to help within bounds
4 = Good policy adherence while still being helpful
5 = Perfect balance of policy compliance and customer advocacy
"""
        }
    
    def evaluate(
        self,
        customer_query: str,
        agent_response: str,
        context: dict | None = None
    ) -> dict:
        """
        Evaluate a customer service response across all criteria.
        
        Args:
            customer_query: The customer's original message
            agent_response: The agent's response to evaluate
            context: Optional context (e.g., company policies, customer history)
        
        Returns:
            Comprehensive evaluation with scores and feedback
        """
        results = {
            "customer_query": customer_query,
            "agent_response": agent_response[:300] + "..." if len(agent_response) > 300 else agent_response,
            "criteria_scores": {},
            "detailed_feedback": {},
            "summary": {}
        }
        
        context_str = ""
        if context:
            context_str = f"\n\nCONTEXT:\n{self._format_context(context)}"
        
        # Evaluate each criterion
        for criterion, rubric in self.rubrics.items():
            score, feedback = self._evaluate_criterion(
                customer_query=customer_query,
                agent_response=agent_response,
                criterion=criterion,
                rubric=rubric,
                context_str=context_str
            )
            results["criteria_scores"][criterion] = score
            results["detailed_feedback"][criterion] = feedback
        
        # Calculate summary
        scores = list(results["criteria_scores"].values())
        results["summary"] = {
            "average_score": sum(scores) / len(scores),
            "lowest_criterion": min(results["criteria_scores"], key=results["criteria_scores"].get),
            "highest_criterion": max(results["criteria_scores"], key=results["criteria_scores"].get),
            "passed": all(s >= 3 for s in scores),
            "needs_improvement": [c for c, s in results["criteria_scores"].items() if s < 3]
        }
        
        return results
    
    def _evaluate_criterion(
        self,
        customer_query: str,
        agent_response: str,
        criterion: str,
        rubric: str,
        context_str: str
    ) -> tuple[int, str]:
        """Evaluate a single criterion using LLM."""
        
        system_prompt = f"""You are an expert evaluator for customer service interactions.
Your task is to evaluate responses based on specific criteria.

{rubric}

Respond in exactly this format:
SCORE: [1-5]
FEEDBACK: [Specific feedback explaining the score with examples from the response]
SUGGESTION: [One specific way to improve, if score < 5]"""

        evaluation_prompt = f"""CUSTOMER QUERY:
{customer_query}

AGENT RESPONSE:
{agent_response}
{context_str}

Evaluate this response for {criterion.upper()}."""

        result = self.llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=evaluation_prompt)
        ])
        
        # Parse response
        content = result.content
        score = self._extract_score(content)
        feedback = self._extract_section(content, "FEEDBACK:")
        suggestion = self._extract_section(content, "SUGGESTION:")
        
        full_feedback = feedback
        if suggestion and score < 5:
            full_feedback += f"\n\nSuggestion: {suggestion}"
        
        return score, full_feedback
    
    def _extract_score(self, content: str) -> int:
        """Extract numeric score from response."""
        for line in content.split('\n'):
            if line.strip().startswith('SCORE:'):
                try:
                    return int(line.split(':')[1].strip().split()[0])
                except (ValueError, IndexError):
                    pass
        return 3
    
    def _extract_section(self, content: str, header: str) -> str:
        """Extract a section from the response."""
        lines = content.split('\n')
        capture = False
        result = []
        
        for line in lines:
            if line.strip().startswith(header):
                capture = True
                result.append(line.split(':', 1)[1].strip() if ':' in line else '')
            elif capture and line.strip().startswith(('SCORE:', 'FEEDBACK:', 'SUGGESTION:')):
                break
            elif capture:
                result.append(line)
        
        return ' '.join(result).strip()
    
    def _format_context(self, context: dict) -> str:
        """Format context dictionary as string."""
        lines = []
        for key, value in context.items():
            lines.append(f"- {key}: {value}")
        return '\n'.join(lines)


# Example usage and test
if __name__ == "__main__":
    evaluator = CustomerServiceEvaluator()
    
    # Test case
    result = evaluator.evaluate(
        customer_query="I ordered a laptop 2 weeks ago and it still hasn't arrived. This is ridiculous! I need it for work!",
        agent_response="I understand how frustrating this must be, especially when you need the laptop for work. Let me look into this right away. I can see your order #12345 was shipped on the 5th. It appears there's been a delay at the carrier's hub. I'm going to: 1) Contact the carrier directly to expedite delivery, 2) Send you tracking updates every 24 hours, and 3) If it's not delivered by Friday, I'll arrange for a replacement with overnight shipping at no extra cost. Does that work for you?",
        context={
            "company_policy": "Can offer replacement or refund after 14 days of delay",
            "customer_tier": "Premium member"
        }
    )
    
    print("Evaluation Results:")
    print(f"Scores: {result['criteria_scores']}")
    print(f"Average: {result['summary']['average_score']:.2f}")
    print(f"Passed: {result['summary']['passed']}")


### Exercise 18.3.2: Comparative Evaluation

In [ ]:
# File: exercise_2_18_3_solution.py
# Description: Exercise 2 Solution - Comparative evaluator for A/B testing

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()


class ComparativeEvaluator:
    """
    Compare two agent responses and determine which is better.
    Useful for A/B testing different agent versions.
    """
    
    def __init__(self, model: str = "gpt-4o"):
        self.llm = ChatOpenAI(model=model, temperature=0)
    
    def compare(
        self,
        question: str,
        response_a: str,
        response_b: str,
        criteria: list[str] | None = None,
        context: str | None = None
    ) -> dict:
        """
        Compare two responses and determine which is better.
        
        Args:
            question: The original question/query
            response_a: First response (e.g., from agent version A)
            response_b: Second response (e.g., from agent version B)
            criteria: Specific criteria to compare on
            context: Additional context for evaluation
        
        Returns:
            Comparison result with winner and reasoning
        """
        criteria = criteria or ["accuracy", "helpfulness", "clarity", "completeness"]
        criteria_str = ", ".join(criteria)
        
        system_prompt = f"""You are an expert evaluator comparing two AI assistant responses.

Your task is to determine which response is better based on these criteria: {criteria_str}

Be objective and thorough. Consider all criteria equally unless one response has a critical flaw.

Respond in exactly this format:
WINNER: [A or B or TIE]
CONFIDENCE: [HIGH, MEDIUM, or LOW]
SUMMARY: [One sentence summary of the decision]

RESPONSE_A_STRENGTHS:
- [strength 1]
- [strength 2]

RESPONSE_A_WEAKNESSES:
- [weakness 1]
- [weakness 2]

RESPONSE_B_STRENGTHS:
- [strength 1]
- [strength 2]

RESPONSE_B_WEAKNESSES:
- [weakness 1]
- [weakness 2]

DETAILED_REASONING:
[Paragraph explaining the decision, comparing specific aspects]"""

        comparison_prompt = f"""QUESTION:
{question}

RESPONSE A:
{response_a}

RESPONSE B:
{response_b}
"""
        if context:
            comparison_prompt += f"\nCONTEXT:\n{context}"
        
        comparison_prompt += f"\n\nCompare these responses based on: {criteria_str}"
        
        result = self.llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=comparison_prompt)
        ])
        
        return self._parse_comparison(result.content, response_a, response_b)
    
    def _parse_comparison(self, content: str, response_a: str, response_b: str) -> dict:
        """Parse the comparison result."""
        result = {
            "response_a_preview": response_a[:150] + "..." if len(response_a) > 150 else response_a,
            "response_b_preview": response_b[:150] + "..." if len(response_b) > 150 else response_b,
            "winner": "TIE",
            "confidence": "MEDIUM",
            "summary": "",
            "response_a_strengths": [],
            "response_a_weaknesses": [],
            "response_b_strengths": [],
            "response_b_weaknesses": [],
            "detailed_reasoning": ""
        }
        
        lines = content.split('\n')
        current_section = None
        
        for line in lines:
            line = line.strip()
            
            if line.startswith('WINNER:'):
                winner = line.split(':')[1].strip().upper()
                if winner in ['A', 'B', 'TIE']:
                    result["winner"] = winner
            elif line.startswith('CONFIDENCE:'):
                conf = line.split(':')[1].strip().upper()
                if conf in ['HIGH', 'MEDIUM', 'LOW']:
                    result["confidence"] = conf
            elif line.startswith('SUMMARY:'):
                result["summary"] = line.split(':', 1)[1].strip()
            elif line.startswith('RESPONSE_A_STRENGTHS:'):
                current_section = "response_a_strengths"
            elif line.startswith('RESPONSE_A_WEAKNESSES:'):
                current_section = "response_a_weaknesses"
            elif line.startswith('RESPONSE_B_STRENGTHS:'):
                current_section = "response_b_strengths"
            elif line.startswith('RESPONSE_B_WEAKNESSES:'):
                current_section = "response_b_weaknesses"
            elif line.startswith('DETAILED_REASONING:'):
                current_section = "detailed_reasoning"
            elif line.startswith('- ') and current_section in [
                "response_a_strengths", "response_a_weaknesses",
                "response_b_strengths", "response_b_weaknesses"
            ]:
                result[current_section].append(line[2:])
            elif current_section == "detailed_reasoning" and line:
                result["detailed_reasoning"] += line + " "
        
        result["detailed_reasoning"] = result["detailed_reasoning"].strip()
        
        return result
    
    def batch_compare(
        self,
        comparisons: list[dict]
    ) -> dict:
        """
        Run multiple comparisons and aggregate results.
        
        Args:
            comparisons: List of dicts with 'question', 'response_a', 'response_b'
        
        Returns:
            Aggregated comparison results
        """
        results = []
        a_wins = 0
        b_wins = 0
        ties = 0
        
        for comp in comparisons:
            result = self.compare(
                question=comp["question"],
                response_a=comp["response_a"],
                response_b=comp["response_b"],
                criteria=comp.get("criteria"),
                context=comp.get("context")
            )
            results.append(result)
            
            if result["winner"] == "A":
                a_wins += 1
            elif result["winner"] == "B":
                b_wins += 1
            else:
                ties += 1
        
        total = len(comparisons)
        
        return {
            "individual_results": results,
            "aggregate": {
                "total_comparisons": total,
                "a_wins": a_wins,
                "b_wins": b_wins,
                "ties": ties,
                "a_win_rate": a_wins / total if total > 0 else 0,
                "b_win_rate": b_wins / total if total > 0 else 0,
                "tie_rate": ties / total if total > 0 else 0,
                "recommendation": self._get_recommendation(a_wins, b_wins, ties)
            }
        }
    
    def _get_recommendation(self, a_wins: int, b_wins: int, ties: int) -> str:
        """Generate recommendation based on results."""
        total = a_wins + b_wins + ties
        if total == 0:
            return "No comparisons to analyze"
        
        a_rate = a_wins / total
        b_rate = b_wins / total
        
        if a_rate > 0.6:
            return "Strong preference for Response A (version A recommended)"
        elif b_rate > 0.6:
            return "Strong preference for Response B (version B recommended)"
        elif a_rate > b_rate + 0.1:
            return "Slight preference for Response A (more testing recommended)"
        elif b_rate > a_rate + 0.1:
            return "Slight preference for Response B (more testing recommended)"
        else:
            return "No clear winner (responses are comparable)"


# Example usage
if __name__ == "__main__":
    evaluator = ComparativeEvaluator()
    
    result = evaluator.compare(
        question="How do I reset my password?",
        response_a="Go to settings and click reset password.",
        response_b="To reset your password: 1) Click your profile icon in the top right, 2) Select 'Settings', 3) Under 'Security', click 'Reset Password', 4) Check your email for a reset link. The link expires in 24 hours. Need help? Reply to this message!",
        criteria=["completeness", "clarity", "helpfulness"]
    )
    
    print(f"Winner: {result['winner']} (Confidence: {result['confidence']})")
    print(f"Summary: {result['summary']}")


### Exercise 18.3.3: Evaluation Dashboard

In [ ]:
# File: exercise_3_18_3_solution.py
# Description: Exercise 3 Solution - Evaluation dashboard with summary reports

from datetime import datetime
from typing import Any


class EvaluationDashboard:
    """
    Generate summary reports from evaluation results.
    Provides insights for debugging and tracking improvements.
    """
    
    def __init__(self, history: list[dict] | None = None):
        """
        Initialize with optional historical data.
        
        Args:
            history: List of past evaluation results for trend analysis
        """
        self.history = history or []
    
    def generate_report(
        self,
        evaluation_results: dict,
        version: str = "current",
        include_details: bool = True
    ) -> str:
        """
        Generate a formatted evaluation report.
        
        Args:
            evaluation_results: Results from an evaluation pipeline run
            version: Version identifier for tracking
            include_details: Whether to include detailed case analysis
        
        Returns:
            Formatted report string
        """
        report_lines = []
        
        # Header
        report_lines.append("=" * 60)
        report_lines.append("AGENT EVALUATION REPORT")
        report_lines.append("=" * 60)
        report_lines.append(f"Version: {version}")
        report_lines.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report_lines.append(f"Total Test Cases: {evaluation_results.get('total_cases', 0)}")
        report_lines.append("")
        
        # Overall Summary
        aggregate = evaluation_results.get("aggregate", {})
        report_lines.append("-" * 40)
        report_lines.append("OVERALL SUMMARY")
        report_lines.append("-" * 40)
        
        pass_rate = aggregate.get("pass_rate", 0) * 100
        avg_score = aggregate.get("average_score", 0) * 100
        
        report_lines.append(f"Pass Rate: {pass_rate:.1f}% ({aggregate.get('passed_count', 0)}/{evaluation_results.get('total_cases', 0)})")
        report_lines.append(f"Average Score: {avg_score:.1f}%")
        report_lines.append(self._get_grade(avg_score))
        report_lines.append("")
        
        # Breakdown by Criterion
        report_lines.append("-" * 40)
        report_lines.append("BREAKDOWN BY CRITERION")
        report_lines.append("-" * 40)
        
        criterion_stats = self._calculate_criterion_stats(evaluation_results.get("cases", []))
        
        for criterion, stats in sorted(criterion_stats.items(), key=lambda x: x[1]["avg"], reverse=True):
            bar = self._score_bar(stats["avg"])
            report_lines.append(f"{criterion:20} {bar} {stats['avg']*100:.1f}% (pass: {stats['pass_rate']*100:.0f}%)")
        
        report_lines.append("")
        
        # Worst Performing Cases
        report_lines.append("-" * 40)
        report_lines.append("CASES NEEDING ATTENTION")
        report_lines.append("-" * 40)
        
        worst_cases = self._get_worst_cases(evaluation_results.get("cases", []), n=5)
        
        if worst_cases:
            for i, case in enumerate(worst_cases, 1):
                report_lines.append(f"\n{i}. Case: {case.get('case_id', 'unknown')}")
                report_lines.append(f"   Score: {case['summary']['average_score']*100:.1f}%")
                report_lines.append(f"   Question: {case.get('question', 'N/A')[:60]}...")
                
                failed = [c for c, e in case.get('evaluations', {}).items() 
                         if not e.get('passed', True)]
                if failed:
                    report_lines.append(f"   Failed criteria: {', '.join(failed)}")
        else:
            report_lines.append("All cases passed! 🎉")
        
        report_lines.append("")
        
        # Trend Analysis
        if self.history:
            report_lines.append("-" * 40)
            report_lines.append("TREND ANALYSIS")
            report_lines.append("-" * 40)
            
            trend = self._analyze_trend()
            report_lines.append(f"Evaluations in history: {len(self.history)}")
            report_lines.append(f"Score trend: {trend['direction']}")
            
            if trend['recent_change'] is not None:
                sign = "+" if trend['recent_change'] > 0 else ""
                report_lines.append(f"Recent change: {sign}{trend['recent_change']*100:.1f}%")
        
        report_lines.append("")
        
        # Detailed Case Analysis (optional)
        if include_details and evaluation_results.get("cases"):
            report_lines.append("-" * 40)
            report_lines.append("DETAILED CASE RESULTS")
            report_lines.append("-" * 40)
            
            for case in evaluation_results["cases"][:10]:  # Limit to first 10
                report_lines.append(f"\nCase: {case.get('case_id', 'unknown')}")
                report_lines.append(f"  Overall: {'PASS' if case['summary']['all_passed'] else 'FAIL'} ({case['summary']['average_score']*100:.1f}%)")
                
                for criterion, eval_data in case.get("evaluations", {}).items():
                    status = "✓" if eval_data.get("passed", True) else "✗"
                    score = eval_data.get("normalized_score", eval_data.get("score", 0))
                    if isinstance(score, float) and score <= 1:
                        score_str = f"{score*100:.0f}%"
                    else:
                        score_str = str(score)
                    report_lines.append(f"  {status} {criterion}: {score_str}")
        
        # Footer
        report_lines.append("")
        report_lines.append("=" * 60)
        report_lines.append("END OF REPORT")
        report_lines.append("=" * 60)
        
        return "\n".join(report_lines)
    
    def _calculate_criterion_stats(self, cases: list[dict]) -> dict:
        """Calculate statistics for each evaluation criterion."""
        criterion_scores = {}
        
        for case in cases:
            for criterion, eval_data in case.get("evaluations", {}).items():
                if criterion not in criterion_scores:
                    criterion_scores[criterion] = {"scores": [], "passed": []}
                
                score = eval_data.get("normalized_score", eval_data.get("score", 0))
                if isinstance(score, (int, float)):
                    if score > 1:  # Assume it's a 1-5 scale
                        score = score / 5.0
                    criterion_scores[criterion]["scores"].append(score)
                    criterion_scores[criterion]["passed"].append(eval_data.get("passed", True))
        
        stats = {}
        for criterion, data in criterion_scores.items():
            scores = data["scores"]
            passed = data["passed"]
            stats[criterion] = {
                "avg": sum(scores) / len(scores) if scores else 0,
                "min": min(scores) if scores else 0,
                "max": max(scores) if scores else 0,
                "pass_rate": sum(passed) / len(passed) if passed else 0
            }
        
        return stats
    
    def _get_worst_cases(self, cases: list[dict], n: int = 5) -> list[dict]:
        """Get the n worst performing cases."""
        sorted_cases = sorted(
            cases,
            key=lambda c: c.get("summary", {}).get("average_score", 1)
        )
        return sorted_cases[:n]
    
    def _get_grade(self, score: float) -> str:
        """Convert score to letter grade with emoji."""
        if score >= 90:
            return "Grade: A 🌟"
        elif score >= 80:
            return "Grade: B ✓"
        elif score >= 70:
            return "Grade: C ⚠️"
        elif score >= 60:
            return "Grade: D ⚠️"
        else:
            return "Grade: F ❌"
    
    def _score_bar(self, score: float, width: int = 20) -> str:
        """Create a visual score bar."""
        filled = int(score * width)
        empty = width - filled
        return f"[{'█' * filled}{'░' * empty}]"
    
    def _analyze_trend(self) -> dict:
        """Analyze score trends from history."""
        if len(self.history) < 2:
            return {"direction": "Not enough data", "recent_change": None}
        
        scores = [r.get("aggregate", {}).get("average_score", 0) for r in self.history]
        
        recent_change = scores[-1] - scores[-2] if len(scores) >= 2 else None
        
        # Simple trend direction
        if len(scores) >= 3:
            recent_avg = sum(scores[-3:]) / 3
            older_avg = sum(scores[:-3]) / (len(scores) - 3) if len(scores) > 3 else scores[0]
            
            if recent_avg > older_avg + 0.05:
                direction = "📈 Improving"
            elif recent_avg < older_avg - 0.05:
                direction = "📉 Declining"
            else:
                direction = "➡️ Stable"
        else:
            direction = "➡️ Stable"
        
        return {
            "direction": direction,
            "recent_change": recent_change
        }


# Example usage
if __name__ == "__main__":
    # Sample evaluation results
    sample_results = {
        "total_cases": 10,
        "aggregate": {
            "pass_rate": 0.8,
            "average_score": 0.75,
            "passed_count": 8,
            "failed_count": 2
        },
        "cases": [
            {
                "case_id": "test_001",
                "question": "How do I reset my password?",
                "summary": {"average_score": 0.9, "all_passed": True},
                "evaluations": {
                    "accuracy": {"normalized_score": 0.9, "passed": True},
                    "helpfulness": {"normalized_score": 0.85, "passed": True}
                }
            },
            {
                "case_id": "test_002",
                "question": "What is your refund policy for digital products?",
                "summary": {"average_score": 0.5, "all_passed": False},
                "evaluations": {
                    "accuracy": {"normalized_score": 0.4, "passed": False},
                    "helpfulness": {"normalized_score": 0.6, "passed": True}
                }
            }
        ]
    }
    
    # Sample history for trend analysis
    history = [
        {"aggregate": {"average_score": 0.65}},
        {"aggregate": {"average_score": 0.70}},
        {"aggregate": {"average_score": 0.75}},
    ]
    
    dashboard = EvaluationDashboard(history=history)
    report = dashboard.generate_report(sample_results, version="v2.1.0")
    print(report)


---
## Next Steps

Return to **Chapter 19: Next Topic**